<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Artificial Intelligence in Finance

## Neural Network Classes

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Activation Functions

In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(suppress=True)

In [ ]:
def activation(x, act='linear', deriv=False):
    if act == 'sigmoid':
        if deriv:
            out = activation(x, 'sigmoid', False)
            return out * (1 - out)
        return 1 / (1 + np.exp(-x))
    elif act == 'relu':
        if deriv:
            return np.where(x > 0, 1, 0)
        return np.maximum(x, 0)
    elif act == 'softplus':
        if deriv:
            return activation(x, act='sigmoid')
        return np.log(1 + np.exp(x))
    elif act == 'linear':
        if deriv:
            return 1
        return x
    else:
        raise ValueError('Activation function not known.')

In [ ]:
x = np.linspace(-1, 1, 20)

In [ ]:
activation(x, 'sigmoid')

In [ ]:
activation(x, 'sigmoid', True)

## Learning &mdash; Simple Neural Network

In [ ]:
class sinn:
    def __init__(self, act='linear', lr=0.01, steps=100,
                 verbose=False, psteps=200):
        self.act = act
        self.lr = lr
        self.steps = steps
        self.verbose = verbose
        self.psteps = psteps
    def forward(self):
        ''' Forward propagation.
        '''
        self.l2 = activation(np.dot(self.l0, self.w), self.act)
    def backward(self):
        ''' Backwards propagation.
        '''
        self.e = self.l2 - self.y
        d = self.e * activation(self.l2, self.act, True)
        u = self.lr * np.dot(self.l0.T, d)
        self.w -= u
    def metrics(self, s):
        ''' Performance metrics.
        '''
        mse = (self.e ** 2).mean()
        acc = float(sum(self.l2.round() == self.y) / len(self.y))
        self.res = self.res.append(
            pd.DataFrame({'mse': mse, 'acc': acc}, index=[s,])
        )
        if s % self.psteps == 0 and self.verbose:
                print(f'step={s:5d} | mse={mse:.6f}')
                print(f'           | acc={acc:.6f}')
    def fit(self, l0, y, steps=None, seed=None):
        ''' Fitting step.
        '''
        self.l0 = l0
        self.y = y
        if steps is None:
            steps = self.steps
        self.res = pd.DataFrame()
        samples, features = l0.shape
        if seed is not None:
            np.random.seed(seed)
        self.w = np.random.random((features, 1))
        for s in range(1, steps + 1):
            self.forward()
            self.backward()
            self.metrics(s)
    def predict(self, X):
        ''' Prediction step.
        '''
        return activation(np.dot(X, self.w), self.act)

### Estimation

In [ ]:
features = 5
samples = 5

In [ ]:
np.random.seed(10)
l0 = np.random.standard_normal((samples, features))
l0

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y = np.random.random((samples, 1))
y

In [ ]:
reg = np.linalg.lstsq(l0, y, rcond=-1)[0]

In [ ]:
reg

In [ ]:
np.allclose(np.dot(l0, reg), y)

In [ ]:
model = sinn(lr=0.015, act='linear', steps=6000,
            verbose=True, psteps=1000)

In [ ]:
%time model.fit(l0, y, seed=100)

In [ ]:
model.predict(l0)

In [ ]:
model.predict(l0) - y

### Classification

In [ ]:
features = 5
samples = 10

In [ ]:
np.random.seed(3)
l0 = np.random.randint(0, 2, (samples, features))
l0

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y = np.random.randint(0, 2, (samples, 1))
y

In [ ]:
model = sinn(lr=0.01, act='sigmoid')

In [ ]:
%time model.fit(l0, y, 4000)

In [ ]:
model.l2

In [ ]:
model.predict(l0).round() == y

In [ ]:
ax = model.res['acc'].plot(figsize=(10, 6),
            title='Prediction Accuracy | Classification')
ax.set(xlabel='steps', ylabel='accuracy');

## Learning &mdash; One Hidden Layer

Shallow neural network = ONE hidden layer = not DEEP neural network.

In [ ]:
class shnn:
    def __init__(self, units=12, act='linear', lr=0.01, steps=100,
                 verbose=False, psteps=200, seed=None):
        self.units = units
        self.act = act
        self.lr = lr
        self.steps = steps
        self.verbose = verbose
        self.psteps = psteps
        self.seed = seed
    def initialize(self):
        ''' Initializes the random weights.
        '''
        if self.seed is not None:
            np.random.seed(self.seed)
        samples, features = self.l0.shape
        self.w0 = np.random.random((features, self.units))
        self.w1 = np.random.random((self.units, 1))
    def forward(self):
        ''' Forward propagation.
        '''
        self.l1 = activation(np.dot(self.l0, self.w0), self.act)
        self.l2 = activation(np.dot(self.l1, self.w1), self.act)
    def backward(self):
        ''' Backward propagation.
        '''
        self.e = self.l2 - self.y
        d2 = self.e * activation(self.l2, self.act, True)
        u2 = self.lr * np.dot(self.l1.T, d2)
        self.w1 -= u2
        e1 = np.dot(d2, self.w1.T)
        d1 = e1 * activation(self.l1, self.act, True)
        u1 = self.lr * np.dot(self.l0.T, d1)
        self.w0 -= u1
    def metrics(self, s):
        ''' Performance metrics.
        '''
        mse = (self.e ** 2).mean()
        acc = float(sum(self.l2.round() == self.y) / len(self.y))
        self.res = self.res.append(
            pd.DataFrame({'mse': mse, 'acc': acc}, index=[s,])
        )
        if s % self.psteps == 0 and self.verbose:
                print(f'step={s:5d} | mse={mse:.5f}')
                print(f'           | acc={acc:.5f}')
    def fit(self, l0, y, steps=None):
        ''' Fitting step.
        '''
        self.l0 = l0
        self.y = y
        if steps is None:
            steps = self.steps
        self.res = pd.DataFrame()
        self.initialize()
        self.forward()
        for s in range(1, steps + 1):
            self.backward()
            self.forward()
            self.metrics(s)
    def predict(self, X):
        ''' Prediction step.
        '''
        l1 = activation(np.dot(X, self.w0), self.act)
        l2 = activation(np.dot(l1, self.w1), self.act)
        return l2

### Estimation

In [ ]:
features = 5
samples = 10

In [ ]:
l0 = np.random.standard_normal((samples, features))

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y = np.random.random((samples, 1))

In [ ]:
reg = np.linalg.lstsq(l0, y, rcond=-1)[0]

In [ ]:
(np.dot(l0, reg)  - y)

In [ ]:
((np.dot(l0, reg)  - y) ** 2).mean()

In [ ]:
model = shnn(lr=0.01, units=16, act='softplus',
             verbose=True, psteps=2000, seed=100)

In [ ]:
%time model.fit(l0, y, 8000)

In [ ]:
model.l2 - y

### Classification

In [ ]:
model = shnn(lr=0.025, act='sigmoid', steps=200,
             verbose=True, psteps=50, seed=100)

In [ ]:
l0.round()

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y.round()

In [ ]:
model.fit(l0.round(), y.round())

In [ ]:
ax = model.res.plot(figsize=(10, 6), secondary_y='mse')
ax.get_legend().set_bbox_to_anchor((0.2, 0.5));

## Financial Data

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
sym = 'EUR='

In [ ]:
data = pd.DataFrame(raw[sym])

In [ ]:
lags = 5
cols = []
data['r'] = np.log(data / data.shift(1))
data['d'] = np.where(data['r'] > 0, 1, 0)
for lag in range(1, lags + 1):
    col = f'lag_{lag}'
    data[col] = data['r'].shift(lag)
    cols.append(col)
data.dropna(inplace=True)
data[cols] = (data[cols] - data[cols].mean()) / data[cols].std()

In [ ]:
data.head()

In [ ]:
model = shnn(lr=0.0001, act='sigmoid', steps=10000,
             verbose=True, psteps=2000, seed=100)

In [ ]:
y = data['d'].values.reshape(-1, 1)

In [ ]:
%time model.fit(data[cols].values, y)

In [ ]:
data['p'] = np.where(model.predict(data[cols]) > 0.5, 1, -1)

In [ ]:
data['p'].value_counts()

In [ ]:
data['s'] = data['p'] * data['r']

In [ ]:
# in-sample results
data[['r', 's']].sum().apply(np.exp)

In [ ]:
# in-sample results
data[['r', 's']].cumsum().apply(np.exp).plot(figsize=(10, 6));

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">